In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt

In [ ]:
ts = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")

In [ ]:
ts.head()

In [ ]:
ts.shape[0]

In [ ]:
def getDistrict(fr, district, state):
    new_fr = fr[(fr.Admin2 == district) & (fr.Province_State == state)]
    return new_fr

def plotDistrict(fr, district, state, base_title = 'Deaths-per-day per Million Population', normalize_per_million = True):
    new_fr = getDistrict(fr, district, state)
    # first date is right after Population
    st_col = fr.columns.get_loc("Population") + 1
    # normalize? 
    population = 1
    if normalize_per_million: 
        population = 1e-6 * new_fr.Population.sum() # sum turns it into a scalar
    
    nums = np.copy(new_fr.iloc[:,st_col:])
    
    # all the plot magic is from Stack Overflow and various other 
    # magic sources / deep pits of iniquity
    xaxis_dates = list(new_fr)[st_col:]
    x = [dt.datetime.strptime(d,'%m/%d/%y').date() for d in xaxis_dates]
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_minor_locator(mdates.WeekdayLocator(byweekday=mdates.SU))
    plt.title("%s : %s, %s" % (base_title, district, state))
    plt.gcf().autofmt_xdate()
    plt.grid()
    
    plt.plot(x, nums[0] / population)


In [ ]:
def makeMovingAverageDailyRate(fr):
    # create a copy of the frame, as we're going to muss it up a bit
    new_fr = fr.copy()
    # The first date in our series is January 22, 2020.
    # We'll start our timeline there.
    st_col = ts.columns.get_loc("Population") + 1

    # strip out all the districts with a zero or NaN population
    new_fr = new_fr[new_fr['Population'] > 0]
    
    # now calculate for each day, the increase in deaths relative 
    # to the number of deaths reported exactly one 
    # week before.  This amounts to an estimate, on that day of the 
    # week, of the weekly death rate.
    # We don't actually want the day-to-day change, as there are substantial swings
    # during the week due to the timing of reports, and the way deaths are recorded.
    # (You aren't officially dead until after someone notices.)
    new_fr.iloc[:,st_col+7:] = np.subtract(ts.iloc[:,st_col+7:], ts.iloc[:,st_col:-7])
    
    # now here is the tricky (and perhaps not-very-statistically-sound) part
    # calculate the rolling average (over the preceeding 7 days) of the "weekly" death rate
    # and divide by 7 to get an estimate of the daily death rate... ???
    # I got a "C" in statistics, and this might be a hint as to why...
    
    # If you are looking for a reason to feel good about this: look at it as a trailing estimate
    # of the daily death rate. 
    new_fr.iloc[:,st_col+7:] = np.divide(new_fr.iloc[:, st_col:].rolling(window=7,axis=1).sum().iloc[:,st_col:], 7)
    return new_fr

### Calculate the 7 day moving average of reported deaths-per-day

Deaths per day is calculated by looking at the moving average of deaths
over a 7 day period.  For each day 'x':
```
deaths_over_previous_week[x] = cumulative_deaths_to_date[x] - cumulative_deaths_to_date[x - 7]
deaths_per_day[x] = average(deaths_over_previous_week[x-7...x])
```
Graphing deaths per day as 
```
deaths_per_day[x] = cumulative_deaths_to_date[x] - cumulative_deaths_to_date[x-1]
```
leads to a graph with clear periodicity, as weekend deaths may be delayed until Monday, and there is
(perhaps no more than anectodal) evidence that they dying sometimes hang on until a particular day of
the week, day of the month, or holiday.  Trailing difference over a specified interval (say quarter to 
corresponding quarter in previous year) is often used to comprehend (and *ignore*) seasonal variations
in sales or revenue, for instance.  That seems like a good idea here.  

Think of it as a primitive low-pass FIR filter, if you like to think that way.

In [ ]:
delta_series = makeMovingAverageDailyRate(ts[ts.iloc[:,-1] > 25])

### Remove statistically small reports
We remove all districts that have reported fewer than 25 deaths by the end of the reporting period. 
These tend to have odd patterns that are made more evident by tiny numbers, and perhaps more probable by limited diagnostic and pathology resources. 
We started with 3261 entries (as of 5/11/20), and finish with 

In [ ]:
delta_series.shape[0]

Thats 272 entries. 

### This is an example of a district that has been discarded from the dataset:

Some districts witness a Lazarus effect. (More likely a death is re-classified.)

In [ ]:
plotDistrict(ts, 'Buncombe', 'North Carolina', "Total Deaths to Date", False)

### This is the deaths-to-date data for Baltimore County, Maryland (including the city of Baltimore)

In [ ]:
getDistrict(ts, "Baltimore", "Maryland")

In [ ]:
fig = plt.figure(figsize=[15,10])
plotDistrict(delta_series, 'Baltimore', 'Maryland')

## New York County (Manhattan)

The death rate in the city of New York has been staggering. However, the number of new deaths reported per day has been declining for several weeks.  At this point (May 12, 2020) the death rate is 200 per million per day. 

The third plot puts this in perspective, by not normalizing to the size of the population. 

In [ ]:
plotDistrict(ts, 'New York', 'New York', "Total Deaths Reported to Date", False)
getDistrict(ts, 'New York', 'New York')

In [ ]:
plotDistrict(delta_series, 'New York', 'New York')

In [ ]:
# totals deaths reported per day, without normalization to population
plotDistrict(delta_series, 'New York', 'New York', "Deaths-per-day", False)

## New York -- Can that possibly be true? 

Let's look at the last week or so of data from the "reported deaths" timeseries

In [ ]:
# take the last 14 days of "total deaths to date" reports 
ny = ts[ts.Admin2 == 'New York'].iloc[:,-14:]
ny

In [ ]:
# calculate the mean difference in the 7-day moving averages of 
# new deaths over the previous week.  (ny.iloc[:, someday] - ny.iloc[:, someday - 7] )
# where the first ':' means "all rows" and the stuff after the ',' means "these columns by column number"
(ny.iloc[:,-7:].sum(axis=1) - ny.iloc[:,-14:-7].sum(axis=1)) / 7

### So the numbers appear to be smashed together correctly. 

### But these numbers smell funny

New York city comprises five boroughs: Manhattan, The Bronx, Brooklyn, Queens, and Staten Island. 
These correspond to the counties of New York, Bronx, Kinks, Queens, and Richmond. 

Of these, only New York county (by its FIPS code) reports data.  I believe that this may be due to a geo-informatics issue that starts with the lat/lon of a reporting district and labels it with the enclosing governmental blob, preference being give to the county level. 

But this doesn't account for a discrepancy between the reported total deaths for "New York" at
20K on 5/11/20 in this dataset, and the NYC reports of about 15K.  A CBS news story (https://www.cbsnews.com/news/coronavirus-nyc-deaths-higher-offical-count-cdc-study/) describes a possible cause. The CDC and NYC may be using different criteria to classify deaths.  The CDC uses "excess deaths" over the expected rate as part of their classification, in combination with death certificate data.  NYC counts deaths if confirmed by lab tests.  As of 5/10 NYC had recorded 5200 deaths that were not confirmed by lab test, but that had COVID-19 listed on the death certificate.  

The numbers in this dataset are derived from CDC statistics. 